In [470]:
# !py -3 -m pip install --upgrade Jinja2==3.0.3

In [471]:
from datetime import datetime, timedelta
import os
import yaml
import json
from jinja2 import Environment, FileSystemLoader
import pandas as pd

In [472]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [473]:
#Write file to local directory
def write_file_local(path,file_string, write_mode):
    
    with open(path, write_mode) as file:
        file.write(file_string)

In [474]:
with open('config\project_config.json') as project_config:
    config = json.load(project_config)

SRC_PROJECT_ID = config['src_project_id']
TGT_PROJECT_ID = config['tgt_project_id']
SRC_DATASET = config['src_dataset']
TGT_DATASET = config['tgt_dataset']
AUDIT_DATASET = config['audit_dataset']
TESTER = config['tester']

In [475]:
#path of file with table name and database name in csv file
yaml_template_path = "YAML_Templates"
lob = "Parallon"
lob_abbr = "PBS"
config_file_path = "config\project_config.json"

In [476]:
#get the DDLs for the tables from teradata 
def gen_testing_script_config():
    dt1 = datetime.now()

    try:
        lob_upper = lob.strip().upper()
        lob_abbr_lower = lob_abbr.strip().lower()

        output_path_folder = "Output_Files\\"
        input_path_folder = "Input_Files\\"
        input_filename = "pbs_testing_data.csv"

        if not os.path.exists(f"{output_path_folder}{lob_upper}"):
            os.makedirs(f"{output_path_folder}{lob_upper}")

        # Jinja2 Environment
        env = Environment(loader = FileSystemLoader(yaml_template_path),   trim_blocks=True, lstrip_blocks=True)     

        # yaml config path
        filename = config_file_path
        config = yaml.load(read_contents_file(filename), Loader=yaml.FullLoader)

        # output_filename = "config_pbs_artiva.yaml"
        

        input_filename_path = f"{input_path_folder}{input_filename}"
        df = pd.read_csv(input_filename_path, index_col=None)
        df.sort_values(by=['Source_System','Table_Pattern'])

        source_system = ""
        run_date = (datetime.now()).strftime('%Y-%m-%d %H:%M')
        tester = config["tester"]
        config.update({"ac_comment":f"Tested by {tester} on {run_date}"})

        for index, row in df.iterrows():
            if (source_system != row['Source_System'].strip().lower()):
                if index != 0:
                    filename = "audit_control.j2"
                    template = env.get_template(filename)    
                    # write_file_local(table_output_path, template.render(config, ac_comment = f"Tested by {tester} on {run_date}"), "a")
                    write_file_local(output_filename_path, template.render(config), "a")
                source_system = row['Source_System'].strip().lower()
                output_filename = f"config_{lob_abbr_lower}_{source_system}.yaml"
                output_filename_path = f"{output_path_folder}{lob_upper}\{output_filename}"
                filename = "projects_datasets.j2"
                template = env.get_template(filename)
                write_file_local(output_filename_path, template.render(config) + "\n", "w")
            table_pattern = row['Table_Pattern'].strip().lower()
            exclude_column_list = row['Exclude_Column_List'].strip().lower()
            order_by_expr = row['Order_By_Expr'].strip().lower()
            table_config = {"table_pattern":table_pattern, "exclude_column_list": exclude_column_list, "order_by_expr": order_by_expr}
            filename = "tables.j2"
            template = env.get_template(filename)
            write_file_local(output_filename_path, template.render(table_config) + "\n", "a")
            if index == len(df) - 1:
                filename = "audit_control.j2"
                template = env.get_template(filename)    
                write_file_local(output_filename_path, template.render(config), "a")

    except Exception as e1:
        print(e1)
        print("LOB : {}".format(lob))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

In [477]:
print("Begin of Processing")

gen_testing_script_config()

print("End of Processing")

Begin of Processing
0:00:00.040002
End of Processing
